In [10]:
# Import necessary libraries
import pandas as pd
import numpy as np
from io import StringIO

df = pd.read_csv("sales.csv")

# Display the original data
print("Original Dataset:")
print(df.head(10))
print(f"\nTotal records: {len(df)}")
print(f"Unique stores: {df['Store_ID'].nunique()}")

Original Dataset:
   Sale_ID        Date  Store_ID  Product_ID  Units
0        1  2017-01-01        24           4      1
1        2  2017-01-01        28           1      1
2        3  2017-01-01         6           8      1
3        4  2017-01-01        48           7      1
4        5  2017-01-01        44          18      1
5        6  2017-01-01         1          31      1
6        7  2017-01-01        40           7      1
7        8  2017-01-01        19           2      1
8        9  2017-01-01        38          21      1
9       10  2017-01-01        21           8      1

Total records: 829262
Unique stores: 50


In [11]:
# Get unique store IDs
unique_stores = df['Store_ID'].unique()
print(f"Unique Store IDs: {sorted(unique_stores)}")

# Get all unique dates and products to ensure complete data coverage
all_dates = sorted(df['Date'].unique())
all_products = sorted(df['Product_ID'].unique())
print(f"Date range: {all_dates[0]} to {all_dates[-1]} ({len(all_dates)} dates)")
print(f"Products: {len(all_products)} unique products")

# Group data by Store_ID and Date, then aggregate units sold by Product_ID
def create_store_datasets():
    store_datasets = {}
    
    for store_id in unique_stores:
        print(f"Processing Store {store_id}...")
        
        # Filter data for current store
        store_data = df[df['Store_ID'] == store_id].copy()
        
        # Group by Date and Product_ID, sum the units (this handles multiple records on same date)
        grouped = store_data.groupby(['Date', 'Product_ID'])['Units'].sum().reset_index()
        
        # Pivot to get Product_IDs as columns
        pivoted = grouped.pivot(index='Date', columns='Product_ID', values='Units')
        
        # Reindex to ensure ALL dates are present, filling missing dates with 0
        pivoted = pivoted.reindex(all_dates, fill_value=0)
        
        # Reindex columns to ensure ALL products are present, filling missing products with 0
        pivoted = pivoted.reindex(columns=all_products, fill_value=0)
        
        # Fill any remaining NaN values with 0 and convert to int
        pivoted = pivoted.fillna(0).astype(int)
        
        # Reset index to make Date a column
        pivoted = pivoted.reset_index()
        
        # Rename columns to have "Product_" prefix for product columns
        new_columns = ['Date'] + [f'Product_{col}' for col in pivoted.columns[1:]]
        pivoted.columns = new_columns
        
        # Verify no missing data
        zero_count = (pivoted.iloc[:, 1:] == 0).sum().sum()
        total_records = len(pivoted) * (len(pivoted.columns) - 1)
        print(f"  Store {store_id}: {len(pivoted)} dates, {zero_count}/{total_records} zero values")
        
        store_datasets[store_id] = pivoted
    
    return store_datasets

# Create datasets for each store
store_datasets = create_store_datasets()

# Display sample data for a few stores
print("\nSample data for each store (showing first 10 dates):")
for store_id in sorted(list(store_datasets.keys())[:3]):  # Show first 3 stores
    print(f"\nStore {store_id}:")
    print(store_datasets[store_id].head(10))
    
    # Show some statistics
    store_data = store_datasets[store_id]
    total_sales = store_data.iloc[:, 1:].sum().sum()
    zero_days = (store_data.iloc[:, 1:].sum(axis=1) == 0).sum()
    print(f"  Total sales across all products: {total_sales}")
    print(f"  Days with zero sales across all products: {zero_days}/{len(store_data)}")

Unique Store IDs: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32), np.int64(33), np.int64(34), np.int64(35), np.int64(36), np.int64(37), np.int64(38), np.int64(39), np.int64(40), np.int64(41), np.int64(42), np.int64(43), np.int64(44), np.int64(45), np.int64(46), np.int64(47), np.int64(48), np.int64(49), np.int64(50)]
Date range: 2017-01-01 to 2018-09-30 (638 dates)
Products: 35 unique products
Processing Store 24...
  Store 24: 638 dates, 20772/22330 zero values
Processing Store 28...
  Store 28: 638 dates, 20526/22330 zero values
Processing Store 6...
  Store 6: 638 dates, 20567/22330 zero values
Proc

In [12]:
# Save each store's data to a separate CSV file
import os

# Create a directory for store CSV files
output_dir = "store_data"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving CSV files for each store...")
for store_id in store_datasets:
    store_dir = f"{output_dir}/store_{store_id}"
    if not os.path.exists(store_dir):
        os.makedirs(store_dir)
    filename = f"{store_dir}/store_{store_id}_data.csv"
    store_datasets[store_id].to_csv(filename, index=False)
    print(f"Saved: {filename}")

print(f"\nTotal CSV files created: {len(store_datasets)}")

Saving CSV files for each store...
Saved: store_data/store_24/store_24_data.csv
Saved: store_data/store_28/store_28_data.csv
Saved: store_data/store_6/store_6_data.csv
Saved: store_data/store_48/store_48_data.csv
Saved: store_data/store_44/store_44_data.csv
Saved: store_data/store_1/store_1_data.csv
Saved: store_data/store_40/store_40_data.csv
Saved: store_data/store_19/store_19_data.csv
Saved: store_data/store_38/store_38_data.csv
Saved: store_data/store_21/store_21_data.csv
Saved: store_data/store_34/store_34_data.csv
Saved: store_data/store_37/store_37_data.csv
Saved: store_data/store_5/store_5_data.csv
Saved: store_data/store_8/store_8_data.csv
Saved: store_data/store_3/store_3_data.csv
Saved: store_data/store_33/store_33_data.csv
Saved: store_data/store_30/store_30_data.csv
Saved: store_data/store_4/store_4_data.csv
Saved: store_data/store_41/store_41_data.csv
Saved: store_data/store_10/store_10_data.csv
Saved: store_data/store_16/store_16_data.csv
Saved: store_data/store_36/store

In [13]:
# Verify data completeness - ensure all stores have the same date range
print("Data Completeness Verification:")
print("=" * 50)

all_dates_expected = len(all_dates)
for store_id in sorted(store_datasets.keys())[:10]:  # Check first 10 stores
    store_data = store_datasets[store_id]
    dates_in_store = len(store_data)
    min_date = store_data['Date'].min()
    max_date = store_data['Date'].max()
    
    # Count zero values per store
    product_columns = [col for col in store_data.columns if col.startswith('Product_')]
    zero_values = (store_data[product_columns] == 0).sum().sum()
    total_values = len(store_data) * len(product_columns)
    
    print(f"Store {store_id:2d}: {dates_in_store} dates ({min_date} to {max_date}), "
          f"{zero_values:,}/{total_values:,} zero values ({zero_values/total_values*100:.1f}%)")

print(f"\nExpected dates per store: {all_dates_expected}")
print(f"All stores have complete date coverage: {all([len(store_datasets[store_id]) == all_dates_expected for store_id in store_datasets])}")

# Show a sample of data with zero values to confirm they're preserved
print(f"\nSample data showing zero values are preserved:")
sample_store = store_datasets[1]  # Store 1
sample_dates = sample_store.head(10)

# Find rows with some zero values to demonstrate
for idx, row in sample_dates.iterrows():
    product_values = row.iloc[1:]  # Skip Date column
    if (product_values == 0).any():
        zero_products = [col for col in product_values.index if product_values[col] == 0]
        print(f"Date {row['Date']}: {len(zero_products)} products with 0 sales (e.g., {zero_products[:3]})")
        break

Data Completeness Verification:
Store  1: 638 dates (2017-01-01 to 2018-09-30), 20,629/22,330 zero values (92.4%)
Store  2: 638 dates (2017-01-01 to 2018-09-30), 20,647/22,330 zero values (92.5%)
Store  3: 638 dates (2017-01-01 to 2018-09-30), 20,577/22,330 zero values (92.1%)
Store  4: 638 dates (2017-01-01 to 2018-09-30), 20,322/22,330 zero values (91.0%)
Store  5: 638 dates (2017-01-01 to 2018-09-30), 20,913/22,330 zero values (93.7%)
Store  6: 638 dates (2017-01-01 to 2018-09-30), 20,567/22,330 zero values (92.1%)
Store  7: 638 dates (2017-01-01 to 2018-09-30), 20,093/22,330 zero values (90.0%)
Store  8: 638 dates (2017-01-01 to 2018-09-30), 20,743/22,330 zero values (92.9%)
Store  9: 638 dates (2017-01-01 to 2018-09-30), 19,736/22,330 zero values (88.4%)
Store 10: 638 dates (2017-01-01 to 2018-09-30), 20,438/22,330 zero values (91.5%)

Expected dates per store: 638
All stores have complete date coverage: True

Sample data showing zero values are preserved:
Date 2017-01-01: 32 prod

In [14]:
# Create aggregate data across all stores for each product
print("Creating global product aggregates...")

# Initialize a dictionary to store global product data
global_product_data = {}

# Get the first store's data to use as a template for dates
first_store_id = list(store_datasets.keys())[0]
template_data = store_datasets[first_store_id][['Date']].copy()

# For each product, aggregate sales across all stores
for product_col in product_columns:
    print(f"Aggregating {product_col}...")
    
    # Initialize with zeros
    global_product_data[product_col] = np.zeros(len(all_dates))
    
    # Sum across all stores for this product
    for store_id, store_data in store_datasets.items():
        global_product_data[product_col] += store_data[product_col].values

# Create the global dataframe
global_df = template_data.copy()
for product_col in product_columns:
    global_df[product_col] = global_product_data[product_col].astype(int)

# Create global directory
global_dir = "global_data"
if not os.path.exists(global_dir):
    os.makedirs(global_dir)

# Save global aggregate data
global_filename = f"{global_dir}/global_product_aggregates.csv"
global_df.to_csv(global_filename, index=False)

print(f"Global product aggregates saved to: {global_filename}")
print(f"Global data shape: {global_df.shape}")
print(f"Date range: {global_df['Date'].min()} to {global_df['Date'].max()}")

# Display sample of global data
print("\nSample of global product aggregates (first 10 dates):")
print(global_df.head(10))

# Show total sales per product across all stores and dates
print("\nTotal sales per product across all stores:")
product_totals = global_df[product_columns].sum().sort_values(ascending=False)
print(product_totals.head(10))

Creating global product aggregates...
Aggregating Product_1...
Aggregating Product_2...
Aggregating Product_3...
Aggregating Product_4...
Aggregating Product_5...
Aggregating Product_6...
Aggregating Product_7...
Aggregating Product_8...
Aggregating Product_9...
Aggregating Product_10...
Aggregating Product_11...
Aggregating Product_12...
Aggregating Product_13...
Aggregating Product_14...
Aggregating Product_15...
Aggregating Product_16...
Aggregating Product_17...
Aggregating Product_18...
Aggregating Product_19...
Aggregating Product_20...
Aggregating Product_21...
Aggregating Product_22...
Aggregating Product_23...
Aggregating Product_24...
Aggregating Product_25...
Aggregating Product_26...
Aggregating Product_27...
Aggregating Product_28...
Aggregating Product_29...
Aggregating Product_30...
Aggregating Product_31...
Aggregating Product_32...
Aggregating Product_33...
Aggregating Product_34...
Aggregating Product_35...
Global product aggregates saved to: global_data/global_produc